# Feature Engineering

## Importación de librerías

In [ ]:
#!pip install geopandas

In [ ]:
import numpy as np
import pandas as pd
import re
from shapely.geometry import Point
import geopandas as gpd

In [ ]:
data = pd.read_csv("./resultados/cabaventa_preproc.csv", index_col = 0)

## Creación de nuevas features a partir de la descripción de las propiedades

Se crea una columna con el título y la descripción de las propiedades para la extracción de features adicionales

In [ ]:
data['texto'] = data['title_cleaned'] + " " + data['description_cleaned']

Detección de propiedades con cochera. Incluye una expresión regular para excluir a aquellos registros en la venta no incluye la cochera.

In [ ]:
pattern_cochera = r"(?<!s\/)(?<!sin )(\d|con|la|las|c\/)?(cochera\.?)(?!s?\s?desde)(?!s?\s?\w*?\s?(op|cort|\d))( fija)?"

x = data['texto'].str.extract(pattern_cochera)

x.bfill(axis=1, inplace=True)
x = x.iloc[:,0]
x.loc[~x.isnull()] = True  # not nan
x.loc[x.isnull()] = False   # nan

data['has_cochera'] = x

Se crea una función de detección de amenities y características que pueden ser relevantes para calcular el precio de una propiedad en el texto libre de la publicación.

In [ ]:
def apply_regex(columna, value):
  pattern_sin = r"((sin|no tiene|no contiene|no hay|no es|no esta|no incluye|no se permite|no se permiten|poca|nula)\s+("+ value +"))"
  pattern_ok = r"("+ value +")"
  # return Falso cuando no hay ninguno de los lugares
  if re.search(pattern_sin, columna, re.M|re.I) is not None:
    return False
  elif re.search(pattern_ok, columna, re.M|re.I) is not None:
    return True
  else:
    return False

In [ ]:
amenities = ['patio', 'jardin', 'balcon', 'terraza', 'parrilla', 'sum', 'pileta|piscina', 'luminoso|luminosidad|mucha luz', 'laundry|lavadero|lavarropas', 'baulera', 'gimnasio|gym', 'seguridad|vigilancia', 'vestidor', 'a estrenar', 
             'pool|ping pong|metegol|microcine', 'portero|porteria|encargado', 'jacuzzi|sauna|solarium|yacuzzi', 'apto profesional', 'amenities', 'por escalera', 'a reciclar|para reciclar', 'categoria|primera linea', 'reciclado|reciclada',
             'en pozo|entrega en|de pozo|emprendimiento', 'tiro balanceado|radiante'] 

for i in amenities:
  col_name = "has_"+ i.replace(' ', '_').split("|")[0]
  data[col_name] = data['texto'].apply(lambda x: apply_regex(str(x), i))

In [ ]:
filter_col = [col for col in data if col.startswith('has')]

for col in filter_col:
  print(col, data[col].mean())

## Feature: Cálculo de distancia al subte

Se carga un csv con las coordenadas geográficas de las estaciones de subte de CABA

In [ ]:
url='https://drive.google.com/uc?id=1oxCEj_enxBVjBsDuvuy5w4HF02HasCWl'
subtes = pd.read_csv(url)
subtes.sample()

Se crea una función para crear un punto con las coordenadas geográficas

In [ ]:
def from_x_y(df, x, y):
    gdf = gpd.GeoDataFrame(df, crs={'init': 'epsg:4326'}, geometry=[Point(xy) for xy in zip(df[x], df[y])])
    return gdf

In [ ]:
caba_geo = from_x_y(data, 'lon', 'lat')
subtes_geo = from_x_y(subtes, 'long', 'lat')

Se realiza una proyección para poder calcular las distancias

In [ ]:
caba_gkba = caba_geo.to_crs(crs = "+proj=tmerc +lat_0=-34.629269 +lon_0=-58.4633 +k=0.9999980000000001 +x_0=100000 +y_0=100000 +ellps=intl +units=m +no_defs")
subtes_gkba = subtes_geo.to_crs(crs = "+proj=tmerc +lat_0=-34.629269 +lon_0=-58.4633 +k=0.9999980000000001 +x_0=100000 +y_0=100000 +ellps=intl +units=m +no_defs")

Se define una función que se queda con la distancia, el nombre y la línea de la estación más cercana

In [ ]:
def distancia_subte(x):
  distancias = []
  for y in range(len(subtes_gkba)):
    est = subtes_gkba.geometry[y]
    dist = x.distance(est)
    distancias.append(dist)
  estacion = np.argmin(distancias)
  return min(distancias), subtes_gkba.iloc[estacion]['estacion'], subtes_gkba.iloc[estacion]['linea']

In [ ]:
caba_gkba['dist_subte'], caba_gkba['estacion_subte_cercana'], caba_gkba['linea_subte_cercana'] = zip(*caba_gkba['geometry'].apply(lambda x: distancia_subte(x)))

In [ ]:
data = data.join(caba_gkba[['dist_subte', 'estacion_subte_cercana', 'linea_subte_cercana']])

## Export

In [ ]:
data.to_csv('resultados/cabaventa_feature.csv')